# 1-0 [Building a Movie Recommendation system | K-Nearest Neighbors](https://www.youtube.com/watch?v=4Ws0oPH350U)

In [1]:
import pandas as pd

In [2]:
movies_raw = pd.read_csv(
    "./data/movies.csv",
    usecols=["movieId", "title"],
    dtype={"movieId": "int32", "title": "str"},
)
movies_raw.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [3]:
movies_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
movieId,9742.0,42200.353623,52160.494854,1.0,3248.25,7300.0,76232.0,193609.0


In [4]:
ratings_raw = pd.read_csv(
    "./data/ratings.csv",
    usecols=["userId", "movieId", "rating"],
    dtype={"userId": "int32", "movieId": "int32", "rating": "float32"},
)
ratings_raw.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
ratings_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,100836.0,326.127564,182.618491,1.0,177.0,325.0,477.0,610.0
movieId,100836.0,19435.295718,35530.987199,1.0,1199.0,2991.0,8122.0,193609.0
rating,100836.0,3.501557,1.042529,0.5,3.0,3.5,4.0,5.0


## Sparse matrix and model fit

In [6]:
from scipy.sparse import csr_matrix

In [7]:
movies_users = ratings_raw.pivot(
    index="movieId", columns="userId", values="rating"
).fillna(0)
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
matrix_movies_users = csr_matrix(movies_users.values)
matrix_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float32'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [13]:
matrix_movies_users.todense()

matrix([[4. , 0. , 0. , ..., 2.5, 3. , 5. ],
        [0. , 0. , 0. , ..., 2. , 0. , 0. ],
        [4. , 0. , 0. , ..., 2. , 0. , 0. ],
        ...,
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

In [15]:
matrix_movies_users.toarray()

array([[4. , 0. , 0. , ..., 2.5, 3. , 5. ],
       [0. , 0. , 0. , ..., 2. , 0. , 0. ],
       [4. , 0. , 0. , ..., 2. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

In [9]:
from sklearn.neighbors import NearestNeighbors

In [10]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [11]:
model_knn.fit(matrix_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

## Recommender function

In [12]:
from fuzzywuzzy import process


def recomender(movie_name, data, model, number_of_recommendations):
    model.fit(data)
    index = process.extractOne(movie_name, movies_raw["title"])[2]
    print(f'Movie selected: {movies_raw["title"][index]}\nIndex: {index}')
    print("Searching for recommendations...")
    distances, indices = model.kneighbors(
        data[index], n_neighbors=number_of_recommendations
    )
    # print(distances, indices)
    for match_index in indices:
        print(movies_raw["title"][match_index].where(match_index != index))


recomender("iron man", matrix_movies_users, model_knn, 20)

Movie selected: Iron Man (2008)
Index: 6743
Searching for recommendations...
6743                                            NaN
7197                                  Garage (2007)
7195                        Merry Madagascar (2009)
7354                             A-Team, The (2010)
6726                         Superhero Movie (2008)
7137                         Thirst (Bakjwi) (2009)
7026                                 Scorpio (1973)
7571                                 Win Win (2011)
3880                  Look Who's Talking Now (1993)
6388    After the Wedding (Efter brylluppet) (2006)
7601                       Idiots and Angels (2008)
6755                Nina's Heavenly Delights (2006)
7022                                   Earth (2007)
7338                          Blue Valentine (2010)
4421                         What's Up, Doc? (1972)
6195                             Silent Hill (2006)
5885                                   Crash (2004)
3740                              Motel